In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [0]:
import warnings
warnings.filterwarnings("ignore")  

import sys
import time

from sklearn.feature_extraction.text import CountVectorizer #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer #For TF-IDF
from gensim.models import Word2Vec                          #For Word2Vec

from sklearn.model_selection import train_test_split
from keras.utils import np_utils

###

import itertools
import os

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_data(file_path):
  data = pd.read_csv(file_path, header = 0)
  return data

In [0]:
seed = 7
np.random.seed(seed)

#dataframes = {
#    "tweets": pd.read_csv("../content/tweets_light.csv")
#}

df = read_data("../content/drive/My Drive/Colab Notebooks/tweets_light.csv")
df = df.astype(str)

#plot distribution of sentiments
#dataframes["sentiment"].value_counts().plot(kind='bar');


In [0]:
X = df["content"]
Y = df["sentiment"]

#print(dataframes["content"])

In [146]:
#split train / test set with shuffling
train_size = int(len(dataframes) * .8)
train_content, test_content, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state = 42)

#print(X_train)
print("Training entries: {}, labels: {}".format(len(train_content), len(train_label)))

Training entries: 32000, labels: 32000


In [0]:
#Tokenizing of the words
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_content) # only fit on train, test should not be fit

In [0]:
#Tokenize the text sets
X_train = tokenize.texts_to_matrix(train_content)
X_test = tokenize.texts_to_matrix(test_content)


#Transform text labels to hot-encoded matrix labels
# angry, happy, neutral, sad
# 1. 0. 0. 0. 
# 0. 1. 0. 0.
# 0. 0. 1. 0.
# 0. 0. 0. 1.
encoder = LabelEncoder()
encoder.fit(train_label)
Y_train = encoder.transform(train_label)
Y_test = encoder.transform(test_label)

num_classes = np.max(Y_train) + 1
Y_train = utils.to_categorical(Y_train, num_classes)
Y_test = utils.to_categorical(Y_test, num_classes)

#print(Y_train)
#print(train_label)

In [150]:
batch_size = 32
epochs = 4

# Build the model
# https://keras.io/getting-started/sequential-model-guide/
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

score = model.evaluate(X_test, Y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

Train on 28800 samples, validate on 3200 samples
Epoch 1/4
28800/28800 [==============================] - 15s 510us/step - loss: 1.1534 - acc: 0.5081 - val_loss: 1.1331 - val_acc: 0.5122
Epoch 2/4
28800/28800 [==============================] - 14s 476us/step - loss: 1.0531 - acc: 0.5637 - val_loss: 1.1384 - val_acc: 0.5159
Epoch 3/4
28800/28800 [==============================] - 14s 471us/step - loss: 0.9996 - acc: 0.5895 - val_loss: 1.1523 - val_acc: 0.5206
Epoch 4/4
8000/8000 [==============================] - 0s 55us/step
Test accuracy: 0.529125
